In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import deepRD
from deepRD.diffusionIntegrators import langevinNoiseSampler, langevin
from deepRD.potentials import harmonic
from deepRD.noiseSampler import noiseSampler
from deepRD.noiseSampler import binnedData_qi, binnedData_ri
import deepRD.tools.trajectoryTools as trajectoryTools
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
localDataDirectory = './data/'

In [ ]:
# Load binning sampling models
binnedDataFilename = localDataDirectory + 'qiBinnedData.pickle'
#binnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
binnedData = pickle.load(open(binnedDataFilename, "rb" ))
parameters = binnedData.parameterDictionary
print(parameters)

In [ ]:
# Define particle list
D = parameters['D']
position = [0,0,0]
velocity = [0,0,0]
mass =  parameters['mass']
particle1 = deepRD.particle(position, D, velocity, mass)
particleList = deepRD.particleList([particle1])

In [ ]:
# Define noise sampler
#nSampler = noiseSampler()
nSampler = noiseSampler(binnedData)

In [ ]:
# Define external potential
kconstant = 0.3
harmonicPotential = harmonic(kconstant)

In [ ]:
# Define integrator
#dt = binnedData.lagTimesteps * parameters['dt'] * parameters['stride'] #wrong
dt = parameters['dt']
integratorStride = 5
tfinal = 10000 #100000
KbT = parameters['KbT']
boxsize = parameters['boxsize']
#integType = "symplecticEuler"
integType = "BAOAB"
diffIntegrator = langevinNoiseSampler(dt, integratorStride, tfinal, nSampler, KbT, 
                                      boxsize, integratorType = integType)
diffIntegrator.setExternalPotential(harmonicPotential)
#diffIntegrator = langevin(dt, integratorStride, tfinal, KbT, boxsize, integratorType = integType)

In [ ]:
t, X, V= diffIntegrator.propagate(particleList)

In [ ]:
# Plot trajectory in x, y and z
particleIndex = 0
#plt.plot(X[:,particleIndex], label = ['x','y','z']);
xpos = X[:,particleIndex,0]
ypos = X[:,particleIndex,1]
zpos = X[:,particleIndex,2]
xvel = V[:,particleIndex,0]
yvel = V[:,particleIndex,1]
zvel = V[:,particleIndex,2]
plt.plot(t,xpos, '-', label = 'x');
plt.plot(t,ypos, '-', label = 'y');
plt.plot(t,zpos, '-', label = 'z');
plt.legend()

In [ ]:
plt.plot(t,xvel, '-', label = 'x');
plt.plot(t,yvel, '-', label = 'y');
plt.plot(t,zvel, '-', label = 'z');
plt.legend()

In [ ]:
numbins=25
#averagepos = (np.array(xpos)+np.array(ypos)+np.array(zpos))/3.0
#plt.hist(averagepos, bins = numbins, density= True, #alpha =0.3)
plt.hist(xpos, bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(ypos, bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(zpos, bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
#plt.xlim([-0.15,0.15])
plt.show()

In [ ]:
numbins=25
#averagepos = (np.array(xpos)+np.array(ypos)+np.array(zpos))/3.0
#plt.hist(averagepos, bins = numbins, density= True, #alpha =0.3)
plt.hist(xvel, bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(yvel, bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(zvel, bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
#plt.xlim([-0.15,0.15])
plt.show()

## Load one benchmark trajectory

In [ ]:
import deepRD.tools.trajectoryTools as trajectoryTools
import os

In [ ]:
parentDirectory = os.environ.get('MSMRD') + '/data/MoriZwanzig/benchmark_onetrajectory/'
fnamebase = parentDirectory + 'simMoriZwanzig_'

In [ ]:
traj = trajectoryTools.loadTrajectory(fnamebase, 0)

In [ ]:
# Plot distributions
xpos_ref = []
ypos_ref = []
zpos_ref = []
xvel_ref = []
yvel_ref = []
zvel_ref = []
numbins = 100
for i in range(len(traj)):
    xpos_ref.append(traj[i][1])
    ypos_ref.append(traj[i][2])
    zpos_ref.append(traj[i][3])
    xvel_ref.append(traj[i][4])
    yvel_ref.append(traj[i][5])
    zvel_ref.append(traj[i][6])
averagepos = (np.array(xpos_ref)+np.array(ypos_ref)+np.array(zpos_ref))/3.0
#plt.hist(averagepos, bins = numbins, density= True, alpha =0.3)
plt.hist(np.array(xpos_ref), bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(np.array(ypos_ref), bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(np.array(zpos_ref), bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
plt.show()

In [ ]:
plt.hist(np.array(xvel_ref), bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(np.array(yvel_ref), bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(np.array(zvel_ref), bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
plt.show()

## Comparison

In [ ]:
numbins=50
plt.hist(np.array(xvel), bins = numbins, density= True, alpha=0.5, label='coarse-grain')
plt.hist(np.array(xvel_ref), bins = numbins, density= True, alpha=0.5, label='reference')
plt.legend(fontsize = 14)
plt.xlabel("velocity")
plt.ylabel("velocity distribution")
#plt.show()
plt.savefig("velocity_test.pdf", bbox_inches='tight')

In [ ]:
numbins=50
plt.hist(np.array(xpos), bins = numbins, density= True, alpha=0.5, label='x coarse-grain')
plt.hist(np.array(xpos_ref), bins = numbins, density= True, alpha=0.5, label='x reference')
plt.legend()
plt.show()